In [1]:
# Import libraries 
import pandas as pd
import pprint as ppr
import matplotlib as pyplot
import pydeck as pdy
import numpy as np
import scipy. stats as st
import warnings 
warnings.filterwarnings("ignore")

In [2]:
# Define input data (previously imported from PostgreSQL)
locations_table_csv = "resources/locations_table.csv"
attendance_table_csv = "resources/attendance_table.csv"

In [3]:
# Read in the data sets
locations_df = pd.read_csv(locations_table_csv, low_memory= False)
attendance_df = pd.read_csv(attendance_table_csv, low_memory = False)

In [4]:
# Display Locations data
locations_df.head()

,District Name,School Name,Organization Type,Organization Code,Address,Town,Zipcode,Phone,PreKindergarten,Kindergarten,...,Grade 6,Grade 7,Grade 8,Grade 9,Grade 10,Grade 11,Grade 12,Student Open Date,Interdistrict Magnet,Location
0,Litchfield School District,Center School,Public Schools,740211,125 West St,Litchfield,06759,860-567-7510,1,1,...,0,0,0,0,0,0,0,1984-07-01T00:00:00.000,0.0,"125 West St\nLitchfield, CT 06759\n(41.7457940..."
1,Norwalk School District,Brien McMahon High School,Public Schools,1036211,300 Highland Avenue,Norwalk,06854,203-852-9488,0,0,...,0,0,0,1,1,1,1,1984-07-01T00:00:00.000,0.0,"300 Highland Avenue\nNorwalk, CT 06854\n(41.08..."
2,Waterbury School District,H. S. Chase School,Public Schools,1510711,40 Woodtick Rd.,Waterbury,06705-2207,203-574-8188,1,1,...,0,0,0,0,0,0,0,1984-07-01T00:00:00.000,0.0,"40 Woodtick Rd.\nWaterbury, CT 06705-2207\n(41..."
3,Bridgeport School District,Jettie S. Tisdale School,Public Schools,152211,250 Hollister Avenue,Bridgeport,06607,203-576-8037,1,1,...,1,1,1,0,0,0,0,1984-07-01T00:00:00.000,0.0,"250 Hollister Avenue\nBridgeport, CT 06607\n(4..."
4,Regional School District 06,Goshen Center School,Regional Schools,2060112,50 North St.,Goshen,06756-1509,860-491-6020,1,1,...,0,0,0,0,0,0,0,2002-07-01T00:00:00.000,0.0,"50 North St.\nGoshen, CT 06756-1509\n(41.83492..."


In [5]:
# Filter Locations to specific columns
columns_to_keep = ["District Name", "School Name", "Organization Type", "Location"]
locations_filtered_df = locations_df[columns_to_keep]
locations_filtered_df.head()

,District Name,School Name,Organization Type,Location
0,Litchfield School District,Center School,Public Schools,"125 West St\nLitchfield, CT 06759\n(41.7457940..."
1,Norwalk School District,Brien McMahon High School,Public Schools,"300 Highland Avenue\nNorwalk, CT 06854\n(41.08..."
2,Waterbury School District,H. S. Chase School,Public Schools,"40 Woodtick Rd.\nWaterbury, CT 06705-2207\n(41..."
3,Bridgeport School District,Jettie S. Tisdale School,Public Schools,"250 Hollister Avenue\nBridgeport, CT 06607\n(4..."
4,Regional School District 06,Goshen Center School,Regional Schools,"50 North St.\nGoshen, CT 06756-1509\n(41.83492..."


In [6]:
# Separate "Location" column
locations_filtered_df[["Street", "City", "Coordinates"]] = locations_filtered_df["Location"].str.split('\n', n=2, expand=True)
locations_filtered_df.head()

,District Name,School Name,Organization Type,Location,Street,City,Coordinates
0,Litchfield School District,Center School,Public Schools,"125 West St\nLitchfield, CT 06759\n(41.7457940...",125 West St,"Litchfield, CT 06759","(41.745794007, -73.194830005)"
1,Norwalk School District,Brien McMahon High School,Public Schools,"300 Highland Avenue\nNorwalk, CT 06854\n(41.08...",300 Highland Avenue,"Norwalk, CT 06854","(41.085690004, -73.433214013)"
2,Waterbury School District,H. S. Chase School,Public Schools,"40 Woodtick Rd.\nWaterbury, CT 06705-2207\n(41...",40 Woodtick Rd.,"Waterbury, CT 06705-2207","(41.552633006, -73.005926969)"
3,Bridgeport School District,Jettie S. Tisdale School,Public Schools,"250 Hollister Avenue\nBridgeport, CT 06607\n(4...",250 Hollister Avenue,"Bridgeport, CT 06607","(41.17816201, -73.160356997)"
4,Regional School District 06,Goshen Center School,Regional Schools,"50 North St.\nGoshen, CT 06756-1509\n(41.83492...",50 North St.,"Goshen, CT 06756-1509","(41.83492398, -73.226575038)"


In [7]:
# Create "Latitude" and "Longitude columns
locations_filtered_df[["Latitude", "Longitude"]] = locations_filtered_df["Coordinates"].str.extract(r'\(([^,]+), ([^,]+)\)')
locations_filtered_df.head()

,District Name,School Name,Organization Type,Location,Street,City,Coordinates,Latitude,Longitude
0,Litchfield School District,Center School,Public Schools,"125 West St\nLitchfield, CT 06759\n(41.7457940...",125 West St,"Litchfield, CT 06759","(41.745794007, -73.194830005)",41.745794007,-73.194830005
1,Norwalk School District,Brien McMahon High School,Public Schools,"300 Highland Avenue\nNorwalk, CT 06854\n(41.08...",300 Highland Avenue,"Norwalk, CT 06854","(41.085690004, -73.433214013)",41.085690004,-73.433214013
2,Waterbury School District,H. S. Chase School,Public Schools,"40 Woodtick Rd.\nWaterbury, CT 06705-2207\n(41...",40 Woodtick Rd.,"Waterbury, CT 06705-2207","(41.552633006, -73.005926969)",41.552633006,-73.005926969
3,Bridgeport School District,Jettie S. Tisdale School,Public Schools,"250 Hollister Avenue\nBridgeport, CT 06607\n(4...",250 Hollister Avenue,"Bridgeport, CT 06607","(41.17816201, -73.160356997)",41.17816201,-73.160356997
4,Regional School District 06,Goshen Center School,Regional Schools,"50 North St.\nGoshen, CT 06756-1509\n(41.83492...",50 North St.,"Goshen, CT 06756-1509","(41.83492398, -73.226575038)",41.83492398,-73.226575038


In [8]:
# Drop unnecessary columns
drop_columns = ["Location", "Coordinates"]
locations_filtered_df = locations_filtered_df.drop(columns = drop_columns)
locations_filtered_df.head()

,District Name,School Name,Organization Type,Street,City,Latitude,Longitude
0,Litchfield School District,Center School,Public Schools,125 West St,"Litchfield, CT 06759",41.745794007,-73.194830005
1,Norwalk School District,Brien McMahon High School,Public Schools,300 Highland Avenue,"Norwalk, CT 06854",41.085690004,-73.433214013
2,Waterbury School District,H. S. Chase School,Public Schools,40 Woodtick Rd.,"Waterbury, CT 06705-2207",41.552633006,-73.005926969
3,Bridgeport School District,Jettie S. Tisdale School,Public Schools,250 Hollister Avenue,"Bridgeport, CT 06607",41.17816201,-73.160356997
4,Regional School District 06,Goshen Center School,Regional Schools,50 North St.,"Goshen, CT 06756-1509",41.83492398,-73.226575038


In [10]:
# Convert lat/lon to numeric
locations_filtered_df["Latitude"] = pd.to_numeric(locations_filtered_df["Latitude"])
locations_filtered_df["Longitude"] = pd.to_numeric(locations_filtered_df["Longitude"])
locations_filtered_df.dtypes

District Name         object
School Name           object
Organization Type     object
Street                object
City                  object
Latitude             float64
Longitude            float64
dtype: object

In [12]:
locations_filtered_df.nunique()

District Name         206
School Name          1218
Organization Type      16
Street               1175
City                  861
Latitude             1103
Longitude            1103
dtype: int64

In [16]:
# Create DataFrame with district name and coordinates
district_coordinates_df = locations_filtered_df[["District Name", "Latitude", "Longitude"]]
district_coordinates_df.head()

,District Name,Latitude,Longitude
0,Litchfield School District,41.745794,-73.194830
1,Norwalk School District,41.085690,-73.433214
2,Waterbury School District,41.552633,-73.005927
3,Bridgeport School District,41.178162,-73.160357
4,Regional School District 06,41.834924,-73.226575


In [18]:
# Keep only one set of coordinates per district
district_coordinates_filtered_df = district_coordinates_df.drop_duplicates(subset = "District Name", keep = "first")
district_coordinates_filtered_df.head()

,District Name,Latitude,Longitude
0,Litchfield School District,41.745794,-73.194830
1,Norwalk School District,41.085690,-73.433214
2,Waterbury School District,41.552633,-73.005927
3,Bridgeport School District,41.178162,-73.160357
4,Regional School District 06,41.834924,-73.226575


In [19]:
district_coordinates_filtered_df.nunique()

District Name    206
Latitude         204
Longitude        204
dtype: int64

In [20]:
# Rename District Name column to match other CSV
district_coordinates_filtered_df=district_coordinates_filtered_df.rename(columns={"District Name":"District name"})
district_coordinates_filtered_df.head()

,District name,Latitude,Longitude
0,Litchfield School District,41.745794,-73.194830
1,Norwalk School District,41.085690,-73.433214
2,Waterbury School District,41.552633,-73.005927
3,Bridgeport School District,41.178162,-73.160357
4,Regional School District 06,41.834924,-73.226575


In [21]:
# Display Attendance data
attendance_df.head()

,District code,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate,Reporting period,Date update
0,00000CT,Connecticut,NaN,All Students,500285,0.9169,496092.0,0.9294,508346.0,0.9479,June 2022,2022-07-22
1,00000CT,Connecticut,Homelessness,Students Experiencing Homelessness,1814,0.8348,1735.0,0.8155,3916.0,0.8884,June 2022,2022-07-22
2,00000CT,Connecticut,Students With Disabilities,Students With Disabilities,78417,0.8899,76487.0,0.8946,80365.0,0.9277,June 2022,2022-07-22
3,00000CT,Connecticut,Free/Reduced Lunch,Free Meal Eligible,168984,0.8851,176225.0,0.8861,193706.0,0.9314,June 2022,2022-07-22
4,00000CT,Connecticut,Free/Reduced Lunch,Reduced Price Meal Eligible,29905,0.9184,30886.0,0.9299,27507.0,0.9518,June 2022,2022-07-22


In [22]:
# Drop unnecessary columns from Attendance
drop_columns_2 = ["District code", "Reporting period", "Date update"]
attendance_filtered_df = attendance_df.drop(columns = drop_columns_2)
attendance_filtered_df.head()

,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate
0,Connecticut,NaN,All Students,500285,0.9169,496092.0,0.9294,508346.0,0.9479
1,Connecticut,Homelessness,Students Experiencing Homelessness,1814,0.8348,1735.0,0.8155,3916.0,0.8884
2,Connecticut,Students With Disabilities,Students With Disabilities,78417,0.8899,76487.0,0.8946,80365.0,0.9277
3,Connecticut,Free/Reduced Lunch,Free Meal Eligible,168984,0.8851,176225.0,0.8861,193706.0,0.9314
4,Connecticut,Free/Reduced Lunch,Reduced Price Meal Eligible,29905,0.9184,30886.0,0.9299,27507.0,0.9518


In [25]:
# Merge Locations and Attendance DataFrames
attendance_merged_df = pd.merge(attendance_filtered_df, district_coordinates_filtered_df, on = 'District name', how = "left")
attendance_merged_df.head(25)

,District name,Category,Student group,2021-2022 student count - year to date,2021-2022 attendance rate - year to date,2020-2021 student count,2020-2021 attendance rate,2019-2020 student count,2019-2020 attendance rate,Latitude,Longitude
0,Connecticut,NaN,All Students,500285,0.9169,496092.0,0.9294,508346.0,0.9479,NaN,NaN
1,Connecticut,Homelessness,Students Experiencing Homelessness,1814,0.8348,1735.0,0.8155,3916.0,0.8884,NaN,NaN
2,Connecticut,Students With Disabilities,Students With Disabilities,78417,0.8899,76487.0,0.8946,80365.0,0.9277,NaN,NaN
3,Connecticut,Free/Reduced Lunch,Free Meal Eligible,168984,0.8851,176225.0,0.8861,193706.0,0.9314,NaN,NaN
4,Connecticut,Free/Reduced Lunch,Reduced Price Meal Eligible,29905,0.9184,30886.0,0.9299,27507.0,0.9518,NaN,NaN
5,Connecticut,Free/Reduced Lunch,Free/Reduced Price Meal Eligible,198889,0.8901,207111.0,0.8927,221213.0,0.9340,NaN,NaN
6,Connecticut,English Learners,English Learners,43571,0.8976,40619.0,0.8948,45413.0,0.9389,NaN,NaN
7,Connecticut,Race/Ethnicity,All other races,48700,0.9314,47339.0,0.9483,47260.0,0.9559,NaN,NaN
8,Connecticut,Race/Ethnicity,Black or African American,63099,0.8941,62267.0,0.8931,64200.0,0.9401,NaN,NaN
9,Connecticut,Race/Ethnicity,Hispanic/Latino of any race,146298,0.8935,138260.0,0.8975,136953.0,0.9362,NaN,NaN


In [28]:
attendance_merged_df.nunique()

District name                                201
Category                                       6
Student group                                 13
2021-2022 student count - year to date      1096
2021-2022 attendance rate - year to date     845
2020-2021 student count                     1104
2020-2021 attendance rate                    980
2019-2020 student count                     1114
2019-2020 attendance rate                    557
Latitude                                     198
Longitude                                    198
dtype: int64

In [ ]:
# Overall -- there are 200 school districts, and 2 of them do no have coordinates listed

In [29]:
attendance_merged_df.to_csv("resources/attendance_and_locations.csv", index=False)